In [78]:
# Write your imports here
import pandas as pd
import itertools as iter
import numpy as np
import re


# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

In this lab, you'll be working with the Coffee Quality Index dataset, located [here](https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi). For convenience (and to save trouble in case you can't download files, or someone uploads a newer version), I've provided the dataset in the `data/` folder. The metadata (description) is at the Kaggle link. For this lab, you'll only need `merged_data_cleaned.csv`, as it is the concatenation of the other two datasets.

In this (and the following labs), you'll get several questions and problems. Do your analysis, describe it, use any tools and plots you wish, and answer. You can create any amount of cells you'd like.

Sometimes, the answers will not be unique, and they will depend on how you decide to approach and solve the problem. This is usual - we're doing science after all!

It's a good idea to save your clean dataset after all the work you've done to it.

### Problem 1. Read the dataset (1 point)
This should be self-explanatory. The first column is the index.

In [22]:
coffee_initial = pd.read_csv("data/merged_data_cleaned.csv")

### Problem 2. Observations and features (1 point)
How many observations are there? How many features? Which features are numerical, and which are categorical?

**Note:** Think about the _meaning_, not the data types. The dataset hasn't been thoroughly cleaned.

In [62]:
def assert_all(func, test_cases: list[tuple[any, any, str]]):
  for (input, expected, message) in test_cases:
    actual = func(input)
    assert actual == expected,  f'{message}. Input: "{input}", epxected: "{expected}", actual: "{actual}"'

test_cases_attribute = "test_cases"

def define_test_cases(func, test_cases: list[tuple[any, any, str]]):
  func.__setattr__(test_cases_attribute, test_cases)

def assert_function_works(func):
  def try_get_name(func): 
    return func.__name__ if callable(func) else func
  assert hasattr(func,test_cases_attribute), f'the provided function "{try_get_name}" has no property {test_cases_attribute}. Consider calling {define_test_cases.__name__} to set them'
  assert_all(func, getattr(func, test_cases_attribute))

In [97]:
(rows_count, columns_count) = coffee_initial.shape

categorical = "categorical"
numerical = "numerical"
other = "other"
numerical_types = [float, int, complex, np.int8, np.int16, np.int32, np.int64 ]

def get_series_type(s: pd.Series):
  categorical_types = [str]
  if (s.dtype in numerical_types):
    return numerical
  if(s.dtype in categorical_types):
    return categorical
  # another try whether all except NaN are strings
  if (all( type(item) == str for item in s.dropna() )):
    return categorical
  return other
  

def get_proposed_features(df: pd.DataFrame):
  return [ (colName, get_series_type(series), series.dtype ) for (colName, series) in [(colName, df[colName]) for colName in df]]
  
def print_proposed_features(df: pd.DataFrame):
  proposed_features = get_proposed_features(df)
  keyfunc =  lambda x: x[1]
  
  for key, group in iter.groupby(sorted(proposed_features, key=keyfunc ), keyfunc ):
    group_as_list = list(group)
    print(f'{key}: {len(group_as_list)}')
    print([ f'{colName}({dtype})' for colName, series_type, dtype in group_as_list])

print(f'observations: {rows_count}')
print(f'features: {columns_count}')
print_proposed_features(coffee_initial)

observations: 1339
features: 44
categorical: 24
['Species(object)', 'Owner(object)', 'Country.of.Origin(object)', 'Farm.Name(object)', 'Lot.Number(object)', 'Mill(object)', 'ICO.Number(object)', 'Company(object)', 'Altitude(object)', 'Region(object)', 'Producer(object)', 'Bag.Weight(object)', 'In.Country.Partner(object)', 'Harvest.Year(object)', 'Grading.Date(object)', 'Owner.1(object)', 'Variety(object)', 'Processing.Method(object)', 'Color(object)', 'Expiration(object)', 'Certification.Body(object)', 'Certification.Address(object)', 'Certification.Contact(object)', 'unit_of_measurement(object)']
numerical: 20
['Unnamed: 0(int64)', 'Number.of.Bags(int64)', 'Aroma(float64)', 'Flavor(float64)', 'Aftertaste(float64)', 'Acidity(float64)', 'Body(float64)', 'Balance(float64)', 'Uniformity(float64)', 'Clean.Cup(float64)', 'Sweetness(float64)', 'Cupper.Points(float64)', 'Total.Cup.Points(float64)', 'Moisture(float64)', 'Category.One.Defects(int64)', 'Quakers(float64)', 'Category.Two.Defects(i

In [ ]:
numerical = "numerical"
other = "other"

def get_series_type(s: pd.Series):
  numerical_types = [float, int, complex, np.int8, np.int16, np.int32, np.int64 ]
  categorical_types = [str]
  if (s.dtype in numerical_types):
    return numerical
  if(s.dtype in categorical_types):
    return categorical
  # another try whether all except NaN are strings
  if (all( type(item) == str for item in s.dropna() )):
    return categorical
  return other
  

def get_proposed_features(df: pd.DataFrame):
  return [ (colName, get_series_type(series), series.dtype ) for (colName, series) in [(colName, df[colName]) for colName in df]]
  
def print_proposed_features(df: pd.DataFrame):
  proposed_features = get_proposed_features(df)
  keyfunc =  lambda x: x[1]
  
  for key, group in iter.groupby(sorted(proposed_features, key=keyfunc ), keyfunc ):
    group_as_list = list(group)
    print(f'{key}: {len(group_as_list)}')
    print([ f'{colName}({dtype})' for colName, series_type, dtype in group_as_list])

print(f'observations: {rows_count}')
print(f'features: {columns_count}')
print_proposed_features(coffee_initial)

### Problem 3. Column manipulation (1 point)
Make the column names more Pythonic (which helps with the quality and... aesthetics). Convert column names to `snake_case`, i.e. `species`, `country_of_origin`, `ico_number`, etc. Try to not do it manually.

In [63]:
def to_snake_case(input: str, separator = '.'):
  if(type(input) != str):
    return input
  stripped_input = input.strip()
  split_values = [ val.strip().lower() for val in stripped_input.split(separator) if len(val.strip()) > 0]
  if (len(split_values) == 1):
    return split_values[0]
  return '_'.join(split_values)

define_test_cases(to_snake_case, [
  (5, 5, 'If nto a string - should not modify the input'),
  ('Petar', 'petar', 'Should lowercase the input'),
  ('Price. noVAT', 'price_novat', 'should sanitize the split parts'),
  ('sales_count','sales_count', 'Should not modify the input'),
  ('  sales_count ','sales_count', 'Should trim leading and trailing spaces'),
])

assert_function_works(to_snake_case)
 
coffee_initial_renames = coffee_initial.rename(to_snake_case, axis='columns')
coffee_initial_renames.columns

Index(['unnamed: 0', 'species', 'owner', 'country_of_origin', 'farm_name',
       'lot_number', 'mill', 'ico_number', 'company', 'altitude', 'region',
       'producer', 'number_of_bags', 'bag_weight', 'in_country_partner',
       'harvest_year', 'grading_date', 'owner_1', 'variety',
       'processing_method', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body',
       'balance', 'uniformity', 'clean_cup', 'sweetness', 'cupper_points',
       'total_cup_points', 'moisture', 'category_one_defects', 'quakers',
       'color', 'category_two_defects', 'expiration', 'certification_body',
       'certification_address', 'certification_contact', 'unit_of_measurement',
       'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters'],
      dtype='object')

### Problem 4. Bag weight (1 point)
What's up with the bag weights? Make all necessary changes to the column values. Don't forget to document your methods and assumptions.

In [66]:
no_measure = 'no_measure'
no_value = 'no_value'

def extract_value_components(input: str) -> tuple[float | None, str]:
  if (type(input) != str):
    return no_measure  
  stripped_input = input.strip()
  split_values = [ val.strip().lower() for val in stripped_input.split(' ') if len(val.strip()) > 0]
  if (len(split_values) == 0):
    return (None, no_value)
  if (len(split_values) == 1):
    return (None, no_measure)
  value = None
  try:
    value = float(split_values[0])
  except ValueError:
    print(f'Provided value {split_values[0]} is nto a float')
  return (value, ' '.join(split_values[1:]))

def extract_measrure(input: str):
  (_, unit) = extract_value_components(input)
  return unit

def extract_value_as_float(input: str):
  (value, _) = extract_value_components(input)
  return value
  

def get_all_measures(series: pd.Series) -> list[tuple[str, int]] :
  series_no_na = series.dropna()
  return [
    (key, len(list(group))) for
    key, group in
    iter.groupby(sorted(series_no_na, key=extract_measrure), extract_measrure)]

bag_weight_all_measures = get_all_measures(coffee_initial_renames.bag_weight)

print('All units')
print(bag_weight_all_measures)

All units
[('kg', 1196), ('kg,lbs', 2), ('lbs', 114), ('no_measure', 27)]


The **bag_weight** series should be weight. We see mixed value with or withut measures. Some are in kg, others are in lbs.
We have values in `kg`, values in `lbs` and values without `unit`.
Our approach would be to convert the data as follows:
1. If unit is `lbs` -> we will convert it using $ f(x)_{x \text{ in lbs}} = 0.45359237*x \text{ kg} $
2. For unit `kg,lbs` or those without `unit` - we will assign NA value to them.

In the end we will remove the unit and cast the column as float

In [68]:
def get_wight_as_float(input: str | None, none_values = ['kg,lbs']) -> float:
  if (input is None):
    return input
  assert type(input) is str, f'Incorrect input param. String or None is required. Provided: ${type(input)}'
  (value, unit) = extract_value_components(input)
  if ([no_measure, no_value].count(unit)):
    return None
  if(none_values.count(unit)):
    return None
  if (unit == 'lbs'):
    return value*0.45359237
  return value

define_test_cases(get_wight_as_float, [
  (None, None, 'Should return None if none i provided'),
  ('', None, 'No value should return None'),
  ('5 ', None, 'No measure should return None'),
  ('15  kg,lbs', None, 'Both units should return None'),
  ('1 lbs', 0.45359237, 'Should convert lbs to kg'),
  ('15.6 kg', 15.6, 'Should just extract the value if unit is kg')
])

assert_function_works(get_wight_as_float)

coffee_initial_renames.bag_weight = coffee_initial_renames.bag_weight.apply(get_wight_as_float)
coffee_initial_renames.bag_weight.describe()


count     1310.000000
mean       182.608718
std       1540.260390
min          0.000000
25%          1.000000
50%         50.000000
75%         69.000000
max      19200.000000
Name: bag_weight, dtype: float64

### Problem 5. Dates (1 point)
This should remind you of problem 4 but it's slightly nastier. Fix the harvest years, document the process.

While you're here, fix the expiration dates, and grading dates. Unlike the other column, these should be dates (`pd.to_datetime()` is your friend).

In [120]:
def get_harvest_year(input: str | None, min = 2000, max = 2050) -> int | None:
  def to_int_year(input: any) -> int | None:
    int_val = int(input)
    return int_val if min <= int_val and int_val <= max else None

  if (input is None):
    return input
  if(numerical_types.count(type(input)) > 0):
    return None if np.isnan(input) else to_int_year(input)
  all_matches = re.findall('\d\d\d\d', input)
  if (len(all_matches) == 0):
    return None
  return to_int_year(all_matches[0])

  
  
assert_all(get_harvest_year,[
  (None, None, 'If none provided, should return None'),
  ('Test', None, 'If no year can be found, return None'),
  ('Martch 2008', 2008, 'If no year can be found, return None'),
  ('Sept 2009 - April 2010', 2009, 'Should return the first met year'),
  ('2012', 2012, 'Should jsut parse the date'),
  ('1567', None, 'Should return None if year is less tahn min'),
  ('3569', None, 'Should return None if year is more than max value'),
  (2012.0, 2012, 'Should convert float to int'),
  (np.NaN, None, 'Should return none if valeu is float Nan')
])

coffee_initial_renames.harvest_year = coffee_initial_renames.harvest_year.apply(get_harvest_year)
coffee_initial_renames.harvest_year.describe()

coffee_initial_renames.expiration = pd.to_datetime(coffee_initial_renames.expiration, format='mixed')
coffee_initial_renames.grading_date = pd.to_datetime(coffee_initial_renames.grading_date, format='mixed')

### Problem 6. Countries (1 point)
How many coffees are there with unknown countries of origin? What can you do about them?

### Problem 7. Owners (1 point)
There are two suspicious columns, named `Owner`, and `Owner.1` (they're likely called something different after you solved problem 3). Do something about them. Is there any link to `Producer`?

### Problem 8. Coffee color by country and continent (1 point)
Create a table which shows how many coffees of each color are there in every country. Leave the missing values as they are.

**Note:** If you ask me, countries should be in rows, I prefer long tables much better than wide ones.

Now do the same for continents. You know what continent each country is located in.

### Problem 9. Ratings (1 point)
The columns `Aroma`, `Flavor`, etc., up to `Moisture` represent subjective ratings. Explore them. Show the means and range; draw histograms and / or boxplots as needed. You can even try correlations if you want. What's up with all those ratings?

### Problem 10. High-level errors (1 point)
Check the countries against region names, altitudes, and companies. Are there any discrepancies (e.g. human errors, like a region not matching the country)? Take a look at the (cleaned) altitudes; there has been a lot of preprocessing done to them. Was it done correctly?

### * Problem 11. Clean and explore at will
The dataset claimed to be clean, but we were able to discover a lot of things to fix and do better.

Play around with the data as much as you wish, and if you find variables to tidy up and clean - by all means, do that!